# Import de bibliotecas


In [ ]:
import pandas as pd
import numpy as np
import requests
import time
import os
import ast
import csv
import sys

# Caminho Google Drive e Chaves API

In [ ]:
#Import do Google Colab
from google.colab import drive
drive.mount('/content/drive')

#Caminho base das pastas
caminho_base = 'SEU_CAMINHO_BASE_AQUI'

#Definição dos caminhos para seleção de tabela:
caminho_tabelas = 'SEU_CAMINHO_TABELAS_AQUI'


#Id e senha de acesso da Twitch e IGDb (mesmo)

#Apenas trocar aqui quando vencer o acesso
client_id = 'SUA_CLIENT_ID_AQUI'
access_token = 'SEU_ACCESS_TOKEN_AQUI' #sem o Bearer

## Explicação dos arquivos (à corrigir em outro momento)


**Top 100 Twitch - Coletado via Power Automate**

top_100_twitch = top_100_twitch.CSV

**Top 100 Viewers - Coletado via Power Automate**

top_100_viewers = top_viewers_twitch.CSV

**Database provisório**

twitch_games_database = twitch_games_database_api.CSV

**Database oficial**

twitch_database = twitch_database_api.CSV


# **Correção futura:**
Verificar melhor utilização dos arquivos database, se há necessidade de serem dois arquivos ou se combinam em 1 arquivo somente.

Alterar código de combinação de dfs conforme necessidade.

# Diretório Google Drive

In [ ]:
#ARQUIVOS

#Top 100 Twitch
top_100_twitch = f"{caminho_base}/top_100_twitch.CSV"

#Top 100 Viewers
top_100_viewers = f"{caminho_base}/top_viewers_twitch.CSV"

#-------------------------------------------------------------------------------------------------------------------------#

#Top 100 Twitch Corrigido
top_100_twitch_corrected = f"{caminho_base}/top_100_twitch_corrected.CSV"

#Database provisório
twitch_games_database = f"{caminho_base}/twitch_games_database_api.CSV"

#Top 100 Viewers Corrigido
top_100_viewers_corrected = f"{caminho_base}/top_viewers_twitch_corrected.CSV"

#Database oficial
twitch_database = f"{caminho_base}/twitch_database_api.CSV"

#Arquivo de Contagem de Views da Twitch
twitch_count_views = f"{caminho_base}/twitch_count_views.CSV"

#-------------------------------------------------------------------------------------------------------------------------#

#IGDB

#Twitch Database IGDB (contém relação de ID Twitch e ID IGDB)
twitch_database_igdb = f"{caminho_base}/twitch_database_twitch_igdb.CSV"

#IGDB Database: contém as informações relacionadas ao ID IGDB
igdb_database = f"{caminho_base}/twitch_database_igdb.CSV"

#-------------------------------------------------------------------------------------------------------------------------#

#Tabelas IGDB

# 1 - Collection
# 2 - Company
# 3 - Date
# 4 - Franchise
# 5 - Genre
# 6 - Platform
# 7 - Rating
# 8 - Summary

# Configurações das tabelas
tabelas_config = {
    1: {
        "name": "Collection",
        "base": "Base/base_collection.CSV",
        "base2": "Não disponível",
        "base3": "Não disponível",
        "dim": "Dim/dim_collection.CSV",
        "dim2": "Não disponível",
        "rel": "Rel/rel_collection.CSV",
        "colunas": "collection_id",
        "colunas_dim": "collection_id",
        "api_fields": "collections", #texto que será enviado para API (sem separação em aspas)
        "url_base_dim": "https://api.igdb.com/v4/collections", # Defina a URL base da API
        "url_base_dim_2": "Não disponível",
        "url_base_dim_3": "Não disponível",
    },
    2: {
        "name": "Company",
        "base": "Base/base_company.CSV",
        "base2": "Base/base_company_2api.CSV",
        "base3": "Não disponível",
        "dim": "Dim/dim_company.CSV",
        "dim2": "Não disponível",
        "rel": "Rel/rel_company.CSV",
        "colunas": "company_id",
        "colunas_dim": "Não disponível",
        "api_fields": "involved_companies", #texto que será enviado para API (sem separação em aspas)
        "url_base_dim": "https://api.igdb.com/v4/involved_companies",
        "url_base_dim_2": "https://api.igdb.com/v4/companies",
        "url_base_dim_3": "Não disponível",
    },
    3: {
        "name": "Date",
        "base": "Base/base_date.CSV",
        "base2": "Não disponível",
        "base3": "Não disponível",
        "dim": "Dim/dim_date.CSV",
        "dim2": "Não disponível",
        "rel": "Rel/rel_date.CSV",
        "colunas": "date_id",
        "colunas_dim": "Não disponível",
        "api_fields": "release_dates", #texto que será enviado para API (sem separação em aspas)
        "url_base_dim": "https://api.igdb.com/v4/release_dates",
        "url_base_dim_2": "Não disponível",
        "url_base_dim_3": "Não disponível",
    },
    4: {
        "name": "Franchise",
        "base": "Base/base_franchise.CSV",
        "base2": "Não disponível",
        "base3": "Não disponível",
        "dim": "Dim/dim_franchise.CSV",
        "dim2": "Não disponível",
        "rel": "Rel/rel_franchise.CSV",
        "colunas": "franchise_id",
        "colunas_dim": "franchise_id",
        "api_fields": "franchises", #texto que será enviado para API (sem separação em aspas)
        "url_base_dim": "https://api.igdb.com/v4/franchises", # Defina a URL base da API
        "url_base_dim_2": "Não disponível",
        "url_base_dim_3": "Não disponível",
    },
    5: {
        "name": "Genre",
        "base": "Base/base_genre.CSV",
        "base2": "Não disponível",
        "base3": "Não disponível",
        "dim": "Dim/dim_genre.CSV",
        "dim2": "Não disponível",
        "rel": "Rel/rel_genre.CSV",
        "colunas": "genre_id",
        "colunas_dim": "genre_id",
        "api_fields": "genres", #texto que será enviado para API (sem separação em aspas)
        "url_base_dim": "https://api.igdb.com/v4/genres", # Defina a URL base da API
        "url_base_dim_2": "Não disponível",
        "url_base_dim_3": "Não disponível",
    },
    6: {
        "name": "Platform",
        "base": "Base/base_platform.CSV",
        "base2": "Não disponível",
        "base3": "Não disponível",
        "dim": "Dim/dim_platform.CSV",
        "dim2": "Não disponível",
        "rel": "Rel/rel_platform.CSV",
        "colunas": "platform_id",
        "colunas_dim": "platform_id",
        "api_fields": "platforms", #texto que será enviado para API (sem separação em aspas)
        "url_base_dim": "https://api.igdb.com/v4/platforms", # Defina a URL base da API
        "url_base_dim_2": "Não disponível",
        "url_base_dim_3": "Não disponível",
    },
    7: {
        "name": "Rating",
        "base": "Base/base_rating.CSV",
        "base2": "Base/base_rating_2api.CSV",
        "base3": "Base/base_rating_3api.CSV",
        "dim": "Dim/dim_rating.CSV",
        "dim2": "Dim/dim_rating_2api.CSV",
        "rel": "Rel/rel_rating.CSV",
        "colunas": "rating_id",
        "colunas_dim": "Não disponível",
        "api_fields": "age_ratings", #texto que será enviado para API (sem separação em aspas)
        "url_base_dim": "https://api.igdb.com/v4/age_ratings",
        "url_base_dim_2": "https://api.igdb.com/v4/age_rating_categories",
        "url_base_dim_3": "https://api.igdb.com/v4/age_rating_content_descriptions_v2",
    },
    8: {
        "name": "Summary - Não disponível",
        "base": "Não disponível",
        "base2": "Não disponível",
        "base3": "Não disponível",
        "dim": "Dim/dim_summary.CSV",
        "dim2": "Não disponível",
        "rel": "Não disponível",
        "colunas": "rating_id",
        "colunas_dim": "Não disponível",
        "api_fields": "age_ratings", #texto que será enviado para API (sem separação em aspas)
        "url_base_dim": "https://api.igdb.com/v4/games",
        "url_base_dim_2": "Não disponível",
        "url_base_dim_3": "Não disponível",
    },
}

#-------------------------------------------------------------------------------------------------------------------------#

# URL APIs


In [ ]:
#URLs das chamadas às APIs

#Documentação geral da Twitch:
"https://dev.twitch.tv/docs/api/"


#URL para pegar informações dos Streams da Twitch (Geral):
#Método GET
#Resposta padrão da API em JSON: "data": [{"id":, "name":, "box_art_url":, "igdb_id"}]
url_base_twitch = "https://api.twitch.tv/helix/games?id="


#URL para pegar informações da IGDB:
#Método POST
#Resposta padrão da API em JSON: "data": [{"id":, "name":, "rating":, "summary":, "websites":[]}]
url_base_igdb = "https://api.igdb.com/v4/games"


# Limpeza Nomes Arquivos

In [ ]:
df_game = pd.read_csv(top_100_twitch, sep=",", encoding='latin-1')

df_game['NomedoJogo'] = df_game['NomedoJogo'].replace({
    'Animals, Aquariums, and Zoos': 'Animals Aquariums and Zoos',
    'Pools, Hot Tubs, and Beaches': 'Pools Hot Tubs and Beaches',
    'Yes, Your Grace': 'Yes Your Grace',
    'Warhammer 40,000: Darktide': 'Warhammer 40000: Darktide',
    'Warhammer 40,000: Space Marine II': 'Warhammer 40000: Space Marine II',
    'Yes, Your Grace: Snowfall': 'Yes Your Grace: Snowfall',
    'Ship, Inc.': 'Ship Inc.',
    "No, I'm Not A Human": "No I'm Not A Human"
})

#Salvar df para csv
df_game.to_csv(top_100_twitch_corrected, index=False, encoding='utf-8-sig')

df_game

In [ ]:
df_viewers = pd.read_csv(top_100_viewers, sep=",", encoding='latin-1')

df_viewers['NomedoJogo'] = df_viewers['NomedoJogo'].replace({
    'Animals, Aquariums, and Zoos': 'Animals Aquariums and Zoos',
    'Pools, Hot Tubs, and Beaches': 'Pools Hot Tubs and Beaches',
    'Yes, Your Grace': 'Yes Your Grace',
    'Warhammer 40,000: Darktide': 'Warhammer 40000: Darktide',
    'Warhammer 40,000: Space Marine II': 'Warhammer 40000: Space Marine II',
    'Yes, Your Grace: Snowfall': 'Yes Your Grace: Snowfall',
    'Ship, Inc.': 'Ship Inc.',
    "No, I'm Not A Human": "No I'm Not A Human"
})

#Salvar df para csv
df_viewers.to_csv(top_100_viewers_corrected, index=False, encoding='utf-8-sig')

df_viewers

# Criação Database Twitch

In [ ]:
#Atualização da entrada de jogos novos no Database prévio pelo Top 100 Games

#Leitura do arquivo Top 100 Twitch e do Database
df_games = pd.read_csv(top_100_twitch_corrected, sep=",", encoding='utf-8-sig')
df_database = pd.read_csv(twitch_database, sep=",", encoding='utf-8-sig')

#Exclui colunas de data e Index - irrelevantes para o Database prévio
df_games = df_games.drop(columns=['Date_GET_Games', 'Index'])

#Remove duplicatas
df_games = df_games.drop_duplicates()

#Para garantir a duplicata, agrupa os jogos por Id, Nome e Image
df_games = df_games.groupby(['IDdoJogo', 'NomedoJogo', 'ImagedoJogo']).size().reset_index(name='Count')

#Remove coluna Count criada no agrupamento
df_games = df_games.drop(columns=['Count'])

#Substitui o texto pelo tamanho das imagens a serem geradas posteriormente
df_games['ImagedoJogo'] = df_games['ImagedoJogo'].str.replace('{width}', '300')
df_games['ImagedoJogo'] = df_games['ImagedoJogo'].str.replace('{height}', '400')

#Para garantir a duplicata, agrupa os jogos por Id e Nome, depois remove coluna gerada
df_database = df_database.groupby(['IDdoJogo', 'NomedoJogo', 'ImagedoJogo']).size().reset_index(name='Count')
df_database = df_database.drop(columns=['Count'])

# Converte IDs para string para evitar problemas nas chamadas das APIs
df_games['IDdoJogo'] = df_games['IDdoJogo'].astype(int).astype(str)
df_database['IDdoJogo'] = df_database['IDdoJogo'].astype(int).astype(str)

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_games, df_database], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='IDdoJogo', keep='first')

#Faz a cópia para evitar perda de informações no df
df_combinado = df_combinado.copy()

#Salvar df para csv
#df_combinado.to_csv(twitch_games_database, index=False, encoding='utf-8-sig')

df_combinado

In [ ]:
#Atualização da entrada de jogos novos no Database prévio pelo Top 100 Viewers
#Top 100 Games e Top 100 Viewers são diferentes.

# Leitura dos CSVs
df_atualizado = pd.read_csv(top_100_viewers_corrected, sep=",", encoding='utf-8-sig')
df_viewers = pd.read_csv(twitch_games_database, sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias
df_atualizado = df_atualizado.drop(columns=['Date_GET_Games', 'Index','Count_views', 'user_id', 'user_name', 'thumbnail_url'])

#Para garantir a duplicata, agrupa os jogos por Id e Nome, depois remove coluna gerada
df_atualizado = df_atualizado.groupby(['IDdoJogo', 'NomedoJogo']).size().reset_index(name='Count')
df_atualizado = df_atualizado.drop(columns=['Count'])

# Adiciona coluna de imagem vazia para consulta depois
df_atualizado['ImagedoJogo'] = ''

# Converte IDs para string para evitar problemas nas chamadas das APIs
df_atualizado['IDdoJogo'] = df_atualizado['IDdoJogo'].astype(int).astype(str)
df_viewers['IDdoJogo'] = df_viewers['IDdoJogo'].astype(int).astype(str)

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_viewers, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='IDdoJogo', keep='first')

#Faz a cópia para evitar perda de informações no df
df_combinado = df_combinado.copy()

#-------------------------------------------------------------------------------------------------------------------------#

# Define a URL base da API (imagem)
url_base = url_base_twitch

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_imagens = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar o box_art_url da API
def get_game_image(game_id):
    #Checa se o Stream já tem imagem ou não (certeza do funcionamento para remoção de Streams duplicados, principalmente os consultados pela API), utilizando o dicionário
    if game_id in cache_imagens:
        return cache_imagens[game_id]

    #Definição da URL com a ID da Stream
    url = f"{url_base}{game_id}"

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {game_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.get(url, headers=headers)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável game_data (dicionário)
            game_data = response.json()

            #Resposta padrão da API em JSON: "data": [{"id":, "name":, "box_art_url":, "igdb_id"}]
            #Verificação da lista "data" no dicionário "game_data"
            if game_data['data']:
                #Salva o box_art_url ao acessar o primeiro item da lista ([0])
                #Caso não encontre, salva como None (Vazio)
                box_art_url = game_data['data'][0].get('box_art_url', None)

                #Checa o resultado do box_art_url
                if box_art_url:
                    print(f"Box Art encontrado para {game_id}: {box_art_url}")

                    #Salva no dicionário cache
                    cache_imagens[game_id] = box_art_url

                    #Retorno da função
                    return box_art_url
                else:
                    print(f"[Aviso] Nenhum box_art_url encontrado para o ID {game_id}")
                    cache_imagens[game_id] = 'NA'  # Armazenar 'NA' se não houver box_art_url
                    return 'NA'

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Nenhum dado retornado pela API para o ID {game_id}")
                cache_imagens[game_id] = 'NA'  # Armazenar 'NA' se não houver dados
                return 'NA'

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {game_id} - Status: {response.status_code}")
            cache_imagens[game_id] = 'Erro'  # Armazenar 'Erro' se a API falhar
            return 'Erro'

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {game_id}: {e}")
        cache_imagens[game_id] = 'Erro'  # Armazenar 'Erro' se houver erro na consulta
        return 'Erro'

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_game_images(df):
    #Copia em um novo df apenas os ID que possuem Imagem vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_imagem = df[df['ImagedoJogo'].isna() | (df['ImagedoJogo'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_imagem.empty:
        print("Nenhum jogo novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_faltando_imagem), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_faltando_imagem.iloc[i:i + batch_size]

        # Para cada linha do batch, pega o ID, e salva o retorno da Função acima
        for idx, row in batch.iterrows():
            game_id = row['IDdoJogo']
            img = get_game_image(game_id)
            df.loc[idx, 'ImagedoJogo'] = img

        #Informa usuário qual lote (inteiro) está sendo processado.
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_imagem) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)
    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_game_images(df_combinado)

#Substitui o texto pelo tamanho das imagens a serem geradas posteriormente
df_combinado['ImagedoJogo'] = df_combinado['ImagedoJogo'].str.replace('{width}', '300')
df_combinado['ImagedoJogo'] = df_combinado['ImagedoJogo'].str.replace('{height}', '400')

# Salva o DataFrame atualizado
#df_combinado.to_csv(twitch_database, index=False, encoding='utf-8-sig')

# Exibe resultado final
df_combinado


In [ ]:
#Preparação do arquivo de contagem de Views
df_atualizado = pd.read_csv (top_100_viewers_corrected, sep=",", encoding='utf-8-sig')

#Remove colunas desnecessárias
df_atualizado = df_atualizado.drop(columns = ['Index', 'user_id', 'user_name', 'thumbnail_url'])

#Transforma ID em inteiro e depois string
df_atualizado['IDdoJogo'] = df_atualizado['IDdoJogo'].fillna(0).astype(int)
df_atualizado['IDdoJogo'] = df_atualizado['IDdoJogo'].astype(str)

#Transformar coluna Date em formato Data mesmo
df_atualizado['Date_GET_Games'] = pd.to_datetime(df_atualizado['Date_GET_Games'])

#Ordena por data
df_atualizado = df_atualizado.sort_values(by='Date_GET_Games')

#Faz o agrupamento de linhas baseadas na Data, ID e Nome, e somar os Views
#Reseta também o index do df criado pelo groupby
df_atualizado = df_atualizado.groupby(['Date_GET_Games','IDdoJogo','NomedoJogo'])['Count_views'].sum().reset_index()

#Ordena por data e por soma de Views (maiores Views para cima)
df_atualizado = df_atualizado.sort_values(by=['Date_GET_Games','Count_views'], ascending=[True, False])

#Criado nova coluna de Index por data, agrupado por data e inicia contagem por 1 a cada nova data
df_atualizado['Index_by_date'] = df_atualizado.groupby('Date_GET_Games').cumcount() + 1

#Reseta o index por precaução de se tornar uma nova coluna
df_atualizado = df_atualizado.reset_index(drop=True)

# Salva o DataFrame atualizado
#df_atualizado.to_csv(twitch_count_views, index=False, encoding='utf-8-sig')

# Exibe resultado final
df_atualizado

# Criação Database IGDB


In [ ]:
#API IGDB - Database IGDB - Consulta ID IGDB

#Leitura do arquivo Twitch Database e IGDB Database
df_base = pd.read_csv(twitch_database_igdb, sep=",", encoding='utf-8-sig')
df_atualizado = pd.read_csv(twitch_database, sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado e renomeia IDdoJogo para ID_Twitch
df_atualizado = df_atualizado.rename(columns={'IDdoJogo': 'ID_Twitch'})

# Converte para string para evitar problemas
df_base['ID_Twitch'] = df_base['ID_Twitch'].astype(int).astype(str)

#Para o df_base, pode ser que tenham ID_IGDB vazios (já consultados e salvos previamente).
#É aplicado a função lambda para não necessitar reconsulta de API, preenchendo o vazio por 'NA'
df_base['ID_IGDB'] = df_base['ID_IGDB'].apply(lambda x: str(x) if pd.notna(x) else 'NA')

#Transforma o ID Twitch em string para a consulta API
df_atualizado['ID_Twitch'] = df_atualizado['ID_Twitch'].astype(int).astype(str)

# Remove duplicados de base e atualizado
df_base = df_base.drop_duplicates(subset='ID_Twitch')
df_atualizado = df_atualizado.drop_duplicates(subset='ID_Twitch')

# Cria o dataframe combinado juntando base + atualizado
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='ID_Twitch', keep='first')

#Faz a cópia para evitar perda de informações no df
df_combinado = df_combinado.copy()


#-------------------------------------------------------------------------------------------------------------------------#

# Define a URL base da API (imagem)
url_base = url_base_twitch

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_igdb_id = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar o box_art_url da API
def get_igdb_id(game_id):
    #Checa se o ID já tem imagem ou não (certeza do funcionamento para remoção de ID duplicados, principalmente os consultados pela API), utilizando o dicionário
    if game_id in cache_igdb_id:
        return cache_igdb_id[game_id]

    #Definição da URL com a ID
    url = f"{url_base}{game_id}"

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {game_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.get(url, headers=headers)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável game_data (dicionário)
            game_data = response.json()

            #Resposta padrão da API em JSON: "data": [{"id":, "name":, "box_art_url":, "igdb_id"}]
            #Verificação da lista "data" no dicionário "game_data"
            if game_data['data']:
                #Salva o igdb_id ao acessar o primeiro item da lista ([0])
                #Caso não encontre, salva como None (Vazio)
                igdb_id = game_data['data'][0].get('igdb_id', None)

                #Checa o resultado do igdb_id
                if igdb_id:
                    print(f"Box Art encontrado para {game_id}: {igdb_id}")

                    #Salva no dicionário cache
                    cache_igdb_id[game_id] = igdb_id

                    #Retorno da função
                    return igdb_id
                else:
                    print(f"[Aviso] Nenhum igdb_id encontrado para o ID {game_id}")
                    cache_igdb_id[game_id] = 'NA'  # Armazenar 'NA' se não houver igdb_id
                    return 'NA'

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Nenhum dado retornado pela API para o ID {game_id}")
                cache_igdb_id[game_id] = 'NA'  # Armazenar 'NA' se não houver dados
                return 'NA'

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {game_id} - Status: {response.status_code}")
            cache_igdb_id[game_id] = 'Erro'  # Armazenar 'Erro' se a API falhar
            return 'Erro'

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {game_id}: {e}")
        cache_igdb_id[game_id] = 'Erro'  # Armazenar 'Erro' se houver erro na consulta
        return 'Erro'

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_igdb_ids(df):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_id = df[df['ID_IGDB'].isna() | (df['ID_IGDB'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_id.empty:
        print("Nenhum jogo novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_faltando_id), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_faltando_id.iloc[i:i + batch_size]

        # Para cada linha do batch, pega o ID, e salva o retorno da Função acima
        for idx, row in batch.iterrows():
            game_id = row['ID_Twitch']
            img = get_igdb_id(game_id)
            df.loc[idx, 'ID_IGDB'] = img

        #Informa usuário qual lote (inteiro) está sendo processado.
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_id) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)
    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_igdb_ids(df_combinado)

#Salvar df para csv
#df_combinado.to_csv(twitch_database_igdb, index=False, encoding='utf-8-sig')

df_combinado


In [ ]:
#API IGDB - Database IGDB - Consulta informações IGDB

#Leitura do arquivo IGDB Database e do arquivo relacionado com ID Twitch
df_atualizado = pd.read_csv(twitch_database_igdb, sep=",", encoding='utf-8-sig')
df_igdb = pd.read_csv(igdb_database, sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado
df_atualizado = df_atualizado.drop(columns=['ID_Twitch', 'NomedoJogo'])

#Para garantir a duplicata, agrupa os jogos por Id, depois remove coluna gerada
df_atualizado = df_atualizado.groupby(['ID_IGDB']).size().reset_index(name='Count')
df_atualizado = df_atualizado.drop(columns=['Count'])

# Adiciona colunas vazias para consulta depois
cols_to_add = ['name', 'rating', 'rating_count', 'aggregated_rating', 'aggregated_rating_count',
               'total_rating', 'total_rating_count']
df_atualizado.loc[:, cols_to_add] = None

# Converte IDs para string para evitar problemas nas chamadas das APIs
df_atualizado['ID_IGDB'] = df_atualizado['ID_IGDB'].astype(int).astype(str)
df_igdb['ID_IGDB'] = df_igdb['ID_IGDB'].astype(int).astype(str)

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_igdb, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='ID_IGDB', keep='first')

#Faz a cópia para evitar perda de informações no df
df_combinado = df_combinado.copy()

# Garante os tipos corretos nas colunas após a junção
df_combinado['name'] = df_combinado['name'].astype('string')
df_combinado['rating'] = df_combinado['rating'].astype('float64')
df_combinado['rating_count'] = df_combinado['rating_count'].astype('Int64')
df_combinado['aggregated_rating'] = df_combinado['aggregated_rating'].astype('float64')
df_combinado['aggregated_rating_count'] = df_combinado['aggregated_rating_count'].astype('Int64')
df_combinado['total_rating'] = df_combinado['total_rating'].astype('float64')
df_combinado['total_rating_count'] = df_combinado['total_rating_count'].astype('Int64')

#-------------------------------------------------------------------------------------------------------------------------#

# Defina a URL base da API
url_base = url_base_igdb

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar as informações da API
def get_game_info(game_id):
    #Checa se o ID já tem imagem ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if game_id in cache_info:
        return cache_info[game_id]

    #Definição do body com a ID
    body = f"""
    where id = {game_id};
    fields name, rating, rating_count, aggregated_rating, aggregated_rating_count, total_rating, total_rating_count;
    limit 1;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {game_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(url_base, headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável game_data (dicionário)
            game_data = response.json()

            #Verificação da lista "data" no dicionário "game_data"
            if game_data:
                #Dicionário Game_info para coletar toda a resposta JSON
                game_info = game_data[0]
                game_info_dict = {
                    'name': game_info.get('name') if game_info.get('name') else 'NA',
                    'rating': game_info.get('rating'),
                    'rating_count': game_info.get('rating_count'),
                    'aggregated_rating': game_info.get('aggregated_rating'),
                    'aggregated_rating_count': game_info.get('aggregated_rating_count'),
                    'total_rating': game_info.get('total_rating'),
                    'total_rating_count': game_info.get('total_rating_count'),
                }
                #Salva no dicionário cache
                cache_info[game_id] = game_info_dict

                #Retorno da função
                return game_info_dict

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Resposta 200, mas sem dados para o ID {game_id}")
                game_info_dict = {
                    'name': 'NA',
                    'rating': None,  # Usando None para valores ausentes numéricos
                    'rating_count': None,
                    'aggregated_rating': None,
                    'aggregated_rating_count': None,
                    'total_rating': None,
                    'total_rating_count': None
                }
                cache_info[game_id] = game_info_dict
                return game_info_dict

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {game_id} - Status: {response.status_code}")
            game_info_dict = {
                'name': 'Erro',
                'rating': None,  # Usando None para valores ausentes numéricos
                'rating_count': None,
                'aggregated_rating': None,
                'aggregated_rating_count': None,
                'total_rating': None,
                'total_rating_count': None
            }
            cache_info[game_id] = game_info_dict
            return game_info_dict

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {game_id}: {e}")
        game_info_dict = {
            'name': 'Erro',
            'rating': None,  # Usando None para valores ausentes numéricos
            'rating_count': None,
            'aggregated_rating': None,
            'aggregated_rating_count': None,
            'total_rating': None,
            'total_rating_count': None
        }
        cache_info[game_id] = game_info_dict
        return game_info_dict

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_game_info(df):
    #Copia em um novo df apenas os ID que possuem name vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_info = df[df['name'].isna() | (df['name'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_info.empty:
        print("Nenhum jogo novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
        #Quantidade de Streams processados por iteração
    batch_size = 5

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_faltando_info), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_faltando_info.iloc[i:i + batch_size]

        # Para cada linha do batch, pega o ID do Stream, e salva o retorno da Função acima
        for idx, row in batch.iterrows():
            game_id = row['ID_IGDB']
            info = get_game_info(game_id)
            for col in cols_to_add:
                df.loc[idx, col] = info.get(col, None)

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_info) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_game_info(df_combinado)

#Salvar df para csv
#df_combinado.to_csv(igdb_database, index=False, encoding='utf-8-sig')


df_combinado


# Tabelas IGDB


In [ ]:
#API IGDB - IGDB - Todas as tabelas

#Opção de selecionar Tabela

#Auxiliar para o While
aux = 0

#Roda função até usuário selecionar tabela disponível.
while aux == 0:

    # Exibir opções para o usuário
    print("Escolha uma das tabelas abaixo:")

    #Exibe os valores no name de cada tabela
    for key, value in tabelas_config.items():
        print(f"{key}: {value['name']}")

    #Recebe valor selecionado pelo usuário
    tabela_id  = int(input("Digite o número da tabela desejada: "))

    #Verifica condição, Tabela 8 não está disponível
    #Se for 8, pede para usuário selecionar outra
    if (tabela_id == 8):
        print("Tabela não disponível. Selecione outra.")

    #Ao escolher uma das opções disponíveis
    else:
        #Salva em config
        config = tabelas_config[tabela_id]
        #Altera valor de aux para sair do loop
        aux = 1

#-------------------------------------------------------------------------------------------------------------------------#

# Leitura dos arquivos IGDB Database e da Tabela selecionada
df_atualizado = pd.read_csv(igdb_database, sep=",", encoding='utf-8-sig')
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["base"]), sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado
df_atualizado = df_atualizado.drop(columns=[
    'name', 'rating', 'rating_count', 'aggregated_rating',
    'aggregated_rating_count', 'total_rating', 'total_rating_count'
])

# Converte para string para evitar problemas
df_base['ID_IGDB'] = df_base['ID_IGDB'].apply(lambda x: str(x) if pd.notna(x) else 'NA')

#Somente na primeira vez que salva
#df_base[config["colunas"]] = ''

#Para o df_base, pode ser que tenham IDs vazios (já consultados e salvos previamente).
#É aplicado a função lambda para não necessitar reconsulta de API, preenchendo o vazio por 'NA'
df_base[config["colunas"]] = df_base[config["colunas"]].apply(lambda x: str(x) if pd.notna(x) else 'NA')

#Transforma o ID_IGDB em string para a consulta API
df_atualizado['ID_IGDB'] = df_atualizado['ID_IGDB'].apply(lambda x: str(x) if pd.notna(x) else 'NA')

# Adiciona colunas vazias para consulta depois
df_atualizado[config["colunas"]] = ''

# Remove duplicados de base e atualizado
df_base = df_base.drop_duplicates(subset='ID_IGDB')
df_atualizado = df_atualizado.drop_duplicates(subset='ID_IGDB')

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='ID_IGDB', keep='first')

#Faz a cópia para evitar perda de informações no df
df_combinado = df_combinado.copy()

#-------------------------------------------------------------------------------------------------------------------------#

# Define a URL base da API (imagem)
url_base = url_base_igdb

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

# Função para pegar as informações da API
def get_api_info(game_id, api_fields):
    #Checa se o ID já tem informação ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if game_id in cache_info:
        return cache_info[game_id]

    #Definição do body da URL com a ID
    body = f"""
    where id = {game_id};
    fields {api_fields};
    limit 1;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {game_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(url_base, headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável game_data (dicionário)
            game_data = response.json()

            # Verificando se 'data' é uma lista não vazia
            if isinstance(game_data, list) and len(game_data) > 0:
                #Salva a informação ao acessar o primeiro item da lista ([0])
                #Caso não encontre, salva como 'NA'
                info = str(game_data[0].get(api_fields) or 'NA')

                #Imprime resultado
                print(f"{api_fields} encontrado para {game_id}: {info}")

                #Salva no dicionário cache
                cache_info[game_id] = info

                #Retorno da função
                return info

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"Nenhum dado retornado para {game_id}")
                return 'NA' # Armazenar 'NA' se não houver informação

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"Erro ao acessar API para {game_id}: {response.status_code}")
            return 'Erro'  # Armazenar 'Erro' se a API falhar

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"Exceção ao consultar API para {game_id}: {e}")
        return 'Erro'  # Armazenar 'Erro' se houver erro na consulta

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_df_colunas(df, coluna_df, coluna_api):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_to_update = df[df[coluna_df].isna() | (df[coluna_df].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_to_update.empty:
        print("Todos os jogos já têm id. Nenhuma chamada de API necessária.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5
    #len(df_to_update)

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_to_update), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_to_update.iloc[i:i + batch_size].copy()

        #Chama função da API, a partir do game id, para os campos na api_fields e armazena na respectiva coluna
        batch[coluna_df] = batch['ID_IGDB'].apply(lambda game_id: get_api_info(game_id, coluna_api))

        #Atualiza o DataFrame original com os novos valores
        df.loc[batch.index, coluna_df] = batch[coluna_df]

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_to_update) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_df_colunas(df_combinado, config["colunas"], config["api_fields"])

#Salvar df para csv
#df_combinado.to_csv(os.path.join(caminho_tabelas, config["base"]), index=False, encoding='utf-8-sig')

df_combinado

In [ ]:
#Tabelas Base para Relacional - não precisa de API, só retorna em linhas separadas as ids

#Opção de selecionar Tabela

#Auxiliar para o While
aux = 0

#Roda função até usuário selecionar tabela disponível.
while aux == 0:

    # Exibir opções para o usuário
    print("Escolha uma das tabelas abaixo:")

    #Exibe os valores no name de cada tabela
    for key, value in tabelas_config.items():
        print(f"{key}: {value['name']}")

    #Recebe valor selecionado pelo usuário
    tabela_id  = int(input("Digite o número da tabela desejada: "))

    #Verifica condição, Tabela 8 não está disponível
    #Se for 8, pede para usuário selecionar outra
    if (tabela_id == 8):
        print("Tabela não disponível. Selecione outra.")

    #Ao escolher uma das opções disponíveis
    else:
        #Salva em config
        config = tabelas_config[tabela_id]
        #Altera valor de aux para sair do loop
        aux = 1

#-------------------------------------------------------------------------------------------------------------------------#

# Leitura dos arquivos
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["base"]), sep=",", encoding='utf-8-sig')

#Para o df_base, pode ser que tenham ID_IGDB vazios (já consultados e salvos previamente).
#É aplicado a função lambda para não necessitar reconsulta de API, preenchendo o vazio por 'NA'
df_base['ID_IGDB'] = df_base['ID_IGDB'].apply(lambda x: str(x) if pd.notna(x) else 'NA')
df_base[config["colunas"]] = df_base[config["colunas"]].apply(lambda x: str(x) if pd.notna(x) else 'NA')

# Remove duplicados de base e atualizado
df_base = df_base.drop_duplicates(subset='ID_IGDB', keep='first')

#Faz a cópia para evitar perda de informações no df
df_base = df_base.copy()

#-------------------------------------------------------------------------------------------------------------------------#

# Função para converter uma string para lista
# Garante que a coluna config["colunas"] esteja no formato correto (lista)
def safe_literal_eval(value):
    try:
        # Converte a string para uma lista
        return ast.literal_eval(value) if isinstance(value, str) else value
    except (ValueError, SyntaxError):
        # Se der erro, retorna como uma lista vazia
        #print(f"[Aviso] Valor inválido para conversão: {value}")
        return []

#-------------------------------------------------------------------------------------------------------------------------#

# Função para expandir a coluna 'id_informação' em várias linhas
def expand_info_rows(df, coluna_df):
    # Cria nova lista de linhas para salvar informação provisóriamente
    expanded_rows = []
    for _, row in df.iterrows():
        # Para cada linha, expandir os valores de coluna_df
        for info in row[coluna_df]:
            expanded_rows.append({
                'ID_IGDB': row['ID_IGDB'],
                coluna_df: info
            })

    # Cria um novo DataFrame com as linhas expandidas
    expanded_df = pd.DataFrame(expanded_rows)
    return expanded_df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada de função
df_base[config["colunas"]] = df_base[config["colunas"]].apply(safe_literal_eval)

# Chamada de função
df_expanded = expand_info_rows(df_base, config["colunas"])

#Salvar df para csv
#df_expanded.to_csv(os.path.join(caminho_tabelas, config["rel"]), index=False, encoding='utf-8-sig')

df_expanded


# Tabelas Dimensionais

In [ ]:
 #IGDB Relacionais para Dimensionais - Collection, Franchise, Genre, Platform

#-------------------------------------------------------------------------------------------------------------------------#

#Opção de selecionar Tabela

#Auxiliar para o While
aux = 0

#Roda função até usuário selecionar tabela disponível.
while aux == 0:

    # Exibir opções para o usuário
    print("Escolha uma das tabelas abaixo:")

    #Exibe os valores no name de cada tabela
    for key, value in tabelas_config.items():
        print(f"{key}: {value['name']}")

    #Recebe valor selecionado pelo usuário
    tabela_id  = int(input("Digite o número da tabela desejada: "))

    #Verifica condição, Tabelas 2, 3, 7 e 8 não estão disponíveis
    #Se for 2, 3, 7 ou 8, pede para usuário selecionar outra
    if (tabela_id == 2 or tabela_id == 3 or tabela_id == 7 or tabela_id == 8):
        print("Tabela não disponível. Selecione outra.")

    #Ao escolher uma das opções disponíveis
    else:
        #Salva em config
        config = tabelas_config[tabela_id]
        #Altera valor de aux para sair do loop
        aux = 1

#-------------------------------------------------------------------------------------------------------------------------#

#Leitura dos arquivos de tabelas relacionais e dimensionais
df_atualizado = pd.read_csv(os.path.join(caminho_tabelas, config["rel"]), sep=",", encoding='utf-8-sig')
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["dim"]), sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado
df_atualizado = df_atualizado.drop(columns=['ID_IGDB'])

# Adiciona coluna de informações vazios para consulta depois
cols_to_add = ['name', 'slug']
df_atualizado.loc[:, cols_to_add] = None

#Somente na primeira vez que salva
#df_base.loc[:, cols_to_add] = None

#Para o df_base, pode ser que tenham IDs vazios (já consultados e salvos previamente).
#É aplicado a função lambda para não necessitar reconsulta de API, preenchendo o vazio por 'NA'
df_base['name'] = df_base['name'].apply(lambda x: str(x) if pd.notna(x) else 'NA')
df_base['slug'] = df_base['slug'].apply(lambda x: str(x) if pd.notna(x) else 'NA')

#Transforma os IDs em string para a consulta API, verificando se está vazio ou não. Caso estiver vazio, mantém vazio.
df_atualizado[config["colunas"]] = df_atualizado[config["colunas"]].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df_base[config["colunas"]] = df_base[config["colunas"]].apply(lambda x: int(x) if pd.notna(x) else np.nan)

# Remove duplicados de base e atualizado
df_base = df_base.drop_duplicates(subset=config["colunas"])
df_atualizado = df_atualizado.drop_duplicates(subset=config["colunas"])

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset=config["colunas"], keep='first')

#Faz a cópia para evitar perda de informações no df
df_combinado = df_combinado.copy()

#Transforma as informações em string para a consulta API
df_combinado[config["colunas"]] = df_combinado[config["colunas"]].apply(lambda x: str(x) if pd.notna(x) else 'NA')

#-------------------------------------------------------------------------------------------------------------------------#

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar as informações da API
def get_info(info_id):
    #Checa se o ID já tem informação ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if info_id in cache_info:
        return cache_info[info_id]

    #Definição do body da URL com a ID
    body = f"""
    where id = {info_id};
    fields name, slug;
    limit 1;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {info_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(config["url_base_dim"], headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável game_data (dicionário)
            info_data = response.json()

            # Verificando se 'data' é uma lista não vazia
            if info_data:
                #Salva a informação ao acessar o primeiro item da lista ([0])
                #Caso não encontre, salva como 'NA'
                id_info = info_data[0]
                id_info_dict = {
                    'name': id_info.get('name') if id_info.get('name') else 'NA',
                    'slug': id_info.get('slug'),
                }

                #Salva no dicionário cache
                cache_info[info_id] = id_info_dict

                #Retorno da função
                return id_info_dict

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Resposta 200, mas sem dados para o ID {info_id}")
                id_info_dict = {
                    'name': 'NA',
                    'slug': 'NA',  # Armazenar 'NA' se não houver informação
                }
                cache_info[info_id] = id_info_dict
                return id_info_dict

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {info_id} - Status: {response.status_code}")
            id_info_dict = {
                'name': 'Erro',  # Armazenar 'Erro' se a API falhar
                'slug': 'NA',  # Armazenar 'NA' se não houver informação
            }
            cache_info[info_id] = id_info_dict
            return id_info_dict

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {info_id}: {e}")
        id_info_dict = {
            'name': 'Erro',  # Armazenar 'Erro' se houver erro na consulta
            'slug': 'NA',  # Armazenar 'NA' se não houver informação
        }
        cache_info[info_id] = id_info_dict
        return id_info_dict

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_info(df, coluna_id):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_info = df[df['name'].isna() | (df['name'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_info.empty:
        print("Nenhum jogo novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5
    #len(df_faltando_info)

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_faltando_info), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_faltando_info.iloc[i:i + batch_size]

        # Para cada linha do batch, pega o ID do Stream, e salva o retorno da Função acima
        for idx, row in batch.iterrows():
            info_id = row[coluna_id]
            info = get_info(info_id)
            for col in cols_to_add:
                df.loc[idx, col] = info.get(col, None)

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_info) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_info(df_combinado, config["colunas"])

#Salvar df para csv
#df_combinado.to_csv(os.path.join(caminho_tabelas, config["dim"]), index=False, encoding='utf-8-sig')

df_combinado


# Tabela Release Date

In [ ]:
#API IGDB - IGDB - Tabela Dimensional Release Dates

#Tabela 3 de Release Dates
tabela_id = 3

#Seleção de Config a partir de tabela 3
config = tabelas_config[tabela_id]

#Leitura dos arquivos de tabelas relacionais e dimensionais
df_atualizado = pd.read_csv(os.path.join(caminho_tabelas, config["rel"]), sep=",", encoding='utf-8-sig')
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["dim"]), sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado
df_atualizado = df_atualizado.drop(columns=['ID_IGDB'])

#Para garantir a duplicata, agrupa os jogos por ID da Data, depois remove coluna gerada
df_atualizado = df_atualizado.groupby(['date_id']).size().reset_index(name='Count')
df_atualizado = df_atualizado.drop(columns=['Count'])

# Adiciona colunas vazias para consulta depois
cols_to_add = ['game', 'human', 'm', 'release_region', 'y', 'platform',]
df_atualizado.loc[:, cols_to_add] = None

# Converte para string para evitar problemas
df_atualizado['date_id'] = df_atualizado['date_id'].astype(int).astype(str)
df_base['date_id'] = df_base['date_id'].astype(int).astype(str)

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='date_id', keep='first')

#Faz a cópia para evitar perda de informações no df
df_combinado = df_combinado.copy()

#Transforma as informações para a consulta API
df_combinado['game'] = df_combinado['game'].astype('Int64')  # Para inteiros com NA
df_combinado['human'] = df_combinado['human'].astype('string')  # ou 'object' para strings
df_combinado['m'] = df_combinado['m'].astype('Int64')  # Para inteiros com NA
df_combinado['release_region'] = df_combinado['release_region'].astype('Int64')  # Para inteiros com NA
df_combinado['y'] = df_combinado['y'].astype('Int64')  # Para inteiros com NA
df_combinado['platform'] = df_combinado['platform'].astype('Int64')  # Para inteiros com NA

#-------------------------------------------------------------------------------------------------------------------------#

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar as informações da API
def get_date_info(date_id):
    #Checa se o ID já tem informação ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if date_id in cache_info:
        return cache_info[date_id]

    #Definição do body da URL com a ID
    body = f"""
    where id = {date_id};
    fields game,human,m,release_region,y,platform;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {date_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(config["url_base_dim"], headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável date_data (dicionário)
            date_data = response.json()

            #Verificação da lista "data" no dicionário "game_data"
            if date_data:
                #Salva a informação ao acessar o primeiro item da lista ([0])
                date_info = date_data[0]

                #Caso não encontre, salva como 'NA'
                date_info_dict = {
                    'game': date_info.get('game'),
                    'human': date_info.get('human') if date_info.get('human') else 'NA',
                    'm': date_info.get('m'),
                    'release_region': date_info.get('release_region'),
                    'y': date_info.get('y'),
                    'platform': date_info.get('platform'),
                }

                #Salva no dicionário cache
                cache_info[date_id] = date_info_dict

                #Retorno da função
                return date_info_dict

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Resposta 200, mas sem dados para o ID {date_id}")
                date_info_dict = {
                    'game': None, # Usando None para valores ausentes numéricos
                    'human': 'NA',
                    'm': None,
                    'release_region': None,
                    'y': None,
                    'platform': None
                }
                cache_info[date_id] = date_info_dict
                return date_info_dict

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {date_id} - Status: {response.status_code}")
            date_info_dict = {
                'game': None, # Usando None para valores ausentes numéricos
                'human': 'Erro',
                'm': None,
                'release_region': None,
                'y': None,
                'platform': None
            }
            cache_info[date_id] = date_info_dict
            return date_info_dict

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {date_id}: {e}")
        date_info_dict = {
            'game': None, # Usando None para valores ausentes numéricos
            'human': 'Erro',
            'm': None,
            'release_region': None,
            'y': None,
            'platform': None
        }
        cache_info[date_id] = date_info_dict
        return date_info_dict

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_date_info(df):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_info = df[df['human'].isna() | (df['human'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_info.empty:
        print("Nenhum date novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5
    #len(df_faltando_info)

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_faltando_info), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_faltando_info.iloc[i:i + batch_size]

        # Para cada linha do batch, pega o ID do Stream, e salva o retorno da Função acima
        for idx, row in batch.iterrows():
            date_id = row['date_id']
            info = get_date_info(date_id)
            for col in cols_to_add:
                df.loc[idx, col] = info.get(col, None)

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_info) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)  # para respeitar limite da API

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_date_info(df_combinado)

#Salvar df para csv
#df_combinado.to_csv(os.path.join(caminho_tabelas, config["dim"]), index=False, encoding='utf-8-sig')

df_combinado

# Tabela Companies


In [ ]:
# API IGDB - IGDB - Tabela Dimensional Companies - Part 2

#Tabela 2 de Companies
tabela_id = 2

#Seleção de Config a partir de tabela 2
config = tabelas_config[tabela_id]

#Leitura dos arquivos de tabelas relacionais e dimensionais
df_atualizado = pd.read_csv(os.path.join(caminho_tabelas, config["rel"]), sep=",", encoding='utf-8-sig')
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["base2"]), sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado
df_atualizado = df_atualizado.drop(columns=['ID_IGDB'])

#Para garantir a duplicata, agrupa os jogos por ID, depois remove coluna gerada
df_atualizado = df_atualizado.groupby(['company_id']).size().reset_index(name='Count')
df_atualizado = df_atualizado.drop(columns=['Count'])

# Adiciona colunas vazias para consulta depois
cols_to_add = ['company', 'developer', 'publisher']
df_atualizado.loc[:, cols_to_add] = None

# Converte para string para evitar problemas
df_atualizado['company_id'] = df_atualizado['company_id'].astype(int).astype(str)
df_base['company_id'] = df_base['company_id'].astype(int).astype(str)

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='company_id', keep='first')

#Faz a cópia para evitar perda de informações no df
df_combinado = df_combinado.copy()

#Transforma as informações para a consulta API
df_combinado['company'] = df_combinado['company'].astype('Int64')  # Para inteiros com NA
df_combinado['developer'] = df_combinado['developer'].astype(bool)  # Para True/False
df_combinado['publisher'] = df_combinado['publisher'].astype(bool)  # Para True/False

#-------------------------------------------------------------------------------------------------------------------------#

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar as informações da API
def get_info(info_id):
    #Checa se o ID já tem informação ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if info_id in cache_info:
        return cache_info[info_id]

    #Definição do body da URL com a ID
    body = f"""
    where id = {info_id};
    fields company,developer,publisher;
    limit 1;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {info_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(config["url_base_dim"], headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável date_data (dicionário)
            info_data = response.json()

            #Verificação da lista "data" no dicionário "game_data"
            if info_data:
                #Salva a informação ao acessar o primeiro item da lista ([0])
                info = info_data[0]

                #Caso não encontre, salva como 'NA'
                info_dict = {
                    'company': info.get('company') if info.get('company') else 'NA',
                    'developer': info.get('developer'),
                    'publisher': info.get('publisher')
                }
                #Salva no dicionário cache
                cache_info[info_id] = info_dict

                #Retorno da função
                return info_dict

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Resposta 200, mas sem dados para o ID {info_id}")
                info_dict = {
                    'company': None, # Usando None para valores ausentes numéricos
                    'developer': False,
                    'publisher': False
                }
                cache_info[info_id] = info_dict
                return info_dict

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {info_id} - Status: {response.status_code}")
            info_dict = {
                'company': None,  # Usando None para valores ausentes numéricos
                'developer': False,
                'publisher': False
            }
            cache_info[info_id] = info_dict
            return info_dict

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {info_id}: {e}")
        info_dict = {
            'company': None,  # Usando None para valores ausentes numéricos
            'developer': False,
            'publisher': False
        }
        cache_info[info_id] = info_dict
        return info_dict

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_info(df):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_info = df[df['company'].isna() | (df['company'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_info.empty:
        print("Nenhum date novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5
    #len(df_faltando_info)

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_faltando_info), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_faltando_info.iloc[i:i + batch_size]
        # Para cada linha do batch, pega o ID do Stream, e salva o retorno da Função acima
        for idx, row in batch.iterrows():
            info_id = row['company_id']
            info = get_info(info_id)
            for col in cols_to_add:
                df.loc[idx, col] = info.get(col, None)

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_info) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)  # para respeitar limite da API

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_info(df_combinado)

#Salvar df para csv
#df_combinado.to_csv(os.path.join(caminho_tabelas, config["base2"]), index=False, encoding='utf-8-sig')

df_combinado

In [ ]:
#API IGDB - IGDB - Tabela Dimensional Companies - Part 3

#Tabela 2 de Companies
#tabela_id = 2

#Seleção de Config a partir de tabela 2
#config = tabelas_config[tabela_id]

#Leitura dos arquivos de tabelas relacionais e dimensionais
df_atualizado = pd.read_csv(os.path.join(caminho_tabelas, config["base2"]), sep=",", encoding='utf-8-sig')
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["dim"]), sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado
df_atualizado = df_atualizado.drop(columns=['company_id', 'developer', 'publisher'])

#Para garantir a duplicata, agrupa os jogos por ID da Data, depois remove coluna gerada
df_atualizado = df_atualizado.groupby(['company']).size().reset_index(name='Count')
df_atualizado = df_atualizado.drop(columns=['Count'])

# Adiciona colunas vazias para consulta depois
cols_to_add = ['country', 'description', 'name', 'slug']
df_atualizado.loc[:, cols_to_add] = None

# Converte para string para evitar problemas
df_atualizado = df_atualizado.rename(columns={'company': 'company_id'})
df_atualizado['company_id'] = df_atualizado['company_id'].astype(int).astype(str)
df_base['company_id'] = df_base['company_id'].astype(int).astype(str)

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='company_id', keep='first')

#Faz a cópia para evitar perda de informações no df
df_combinado = df_combinado.copy()

#Transforma as informações para a consulta API
df_combinado['country'] = df_combinado['country'].astype('Int64')  # Para inteiros com NA
df_combinado['description'] = df_combinado['description'].astype('string')  # ou 'object' para strings
df_combinado['name'] = df_combinado['name'].astype('string')  # ou 'object' para strings
df_combinado['slug'] = df_combinado['slug'].astype('string')  # ou 'object' para strings

#-------------------------------------------------------------------------------------------------------------------------#

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar as informações da API
def get_info(info_id):
    #Checa se o ID já tem informação ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if info_id in cache_info:
        return cache_info[info_id]

    #Definição do body da URL com a ID
    body = f"""
    where id = {info_id};
    fields country,description,name,slug;
    limit 1;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {info_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(config["url_base_dim_2"], headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável date_data (dicionário)
            info_data = response.json()

            #Verificação da lista "data" no dicionário "game_data"
            if info_data:
                #Salva a informação ao acessar o primeiro item da lista ([0])
                info = info_data[0]

                #Caso não encontre, salva como 'NA'
                info_dict = {
                    'country': info.get('country'),
                    'description': info.get('description'),
                    'name': info.get('name') if info.get('name') else 'NA',
                    'slug': info.get('slug')
                }

                #Salva no dicionário cache
                cache_info[info_id] = info_dict

                #Retorno da função
                return info_dict

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Resposta 200, mas sem dados para o ID {info_id}")
                info_dict = {
                    'country': None, # Usando None para valores ausentes numéricos
                    'description': None,
                    'name': 'NA',
                    'slug': None
                }
                cache_info[info_id] = info_dict
                return info_dict

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {info_id} - Status: {response.status_code}")
            info_dict = {
                'country': None,  # Usando None para valores ausentes numéricos
                'description': None,
                'name': 'Erro',
                'slug': None
            }
            cache_info[info_id] = info_dict
            return info_dict

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {info_id}: {e}")
        info_dict = {
            'country': None,  # Usando None para valores ausentes numéricos
            'description': None,
            'name': 'Erro',
            'slug': None
        }
        cache_info[info_id] = info_dict
        return info_dict

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_info(df):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_info = df[df['name'].isna() | (df['name'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_info.empty:
        print("Nenhum company novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5
    #len(df_faltando_info)

    # Para cada linha do batch, pega o ID do Stream, e salva o retorno da Função acima
    for i in range(0, len(df_faltando_info), batch_size):
        batch = df_faltando_info.iloc[i:i + batch_size]
        for idx, row in batch.iterrows():
            info_id = row['company_id']
            info = get_info(info_id)
            for col in cols_to_add:
                df.loc[idx, col] = info.get(col, None)

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_info) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)  # para respeitar limite da API

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_info(df_combinado)

#Salvar df para csv
#df_combinado.to_csv(os.path.join(caminho_tabelas, config["dim"]), index=False, encoding='utf-8-sig')

df_combinado

# Tabela Rating

In [ ]:
# API IGDB - IGDB - Tabela Dimensional Age Rating - Part 2

#Tabela 7 de Release Dates
tabela_id = 7

#Seleção de Config a partir de tabela 7
config = tabelas_config[tabela_id]

#Leitura dos arquivos de tabelas relacionais e dimensionais
df_atualizado = pd.read_csv(os.path.join(caminho_tabelas, config["rel"]), sep=",", encoding='utf-8-sig')
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["base2"]), sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado
df_atualizado = df_atualizado.drop(columns=['ID_IGDB'])

#Para garantir a duplicata, agrupa os jogos por ID da Data, depois remove coluna gerada
df_atualizado = df_atualizado.groupby(['rating_id']).size().reset_index(name='Count')
df_atualizado = df_atualizado.drop(columns=['Count'])

# Adiciona colunas vazias para consulta depois
cols_to_add = ['rating_category', 'rating_content_descriptions']
df_atualizado.loc[:, cols_to_add] = None

# Converte para string para evitar problemas
df_atualizado['rating_id'] = df_atualizado['rating_id'].astype(int).astype(str)
df_base['rating_id'] = df_base['rating_id'].astype(int).astype(str)
df_base['rating_content_descriptions'] = df_base['rating_content_descriptions'].apply(lambda x: str(x) if pd.notna(x) else 'NA')

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='rating_id', keep='first')

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = df_combinado.copy()

#Transforma as informações para a consulta API
df_combinado['rating_category'] = df_combinado['rating_category'].astype('Int64')  # Para inteiros com NA
df_combinado['rating_content_descriptions'] = df_combinado['rating_content_descriptions'].astype('string')  # ou 'object' para strings

#-------------------------------------------------------------------------------------------------------------------------#

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar as informações da API
def get_info(info_id):
    #Checa se o ID já tem informação ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if info_id in cache_info:
        return cache_info[info_id]

    #Definição do body da URL com a ID
    body = f"""
    where id = {info_id};
    fields rating_category,rating_content_descriptions;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {info_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(config["url_base_dim"], headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável info_data (dicionário)
            info_data = response.json()

            #Verificação da lista "data" no dicionário "info_data"
            if info_data:
                #Salva a informação ao acessar o primeiro item da lista ([0])
                info = info_data[0]

                #Caso não encontre, salva como 'NA'
                info_dict = {
                    'rating_category': info.get('rating_category'),
                    'rating_content_descriptions': str(info.get('rating_content_descriptions')) if info.get('rating_content_descriptions') not in [None, []] else 'NA'
                }

                #Salva no dicionário cache
                cache_info[info_id] = info_dict

                #Retorno da função
                return info_dict

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Resposta 200, mas sem dados para o ID {info_id}")
                info_dict = {
                    'rating_content_descriptions': 'NA', # Armazenar 'NA' se não houver informação
                    'rating_category': None
                }
                cache_info[info_id] = info_dict
                return info_dict

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {info_id} - Status: {response.status_code}")
            info_dict = {
                'rating_content_descriptions': 'Erro',  # Armazenar 'Erro' se a API falhar
                'rating_category': None
            }
            cache_info[info_id] = info_dict
            return info_dict

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {info_id}: {e}")
        info_dict = {
            'rating_content_descriptions': 'Erro',  # Armazenar 'Erro' se houver erro na consulta
            'rating_category': None
        }
        cache_info[info_id] = info_dict
        return info_dict

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_info(df):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_info = df[df['rating_content_descriptions'].isna() | (df['rating_content_descriptions'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_info.empty:
        print("Nenhum date novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5
    #len(df_faltando_info)

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_faltando_info), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_faltando_info.iloc[i:i + batch_size]

        # Para cada linha do batch, pega o ID do Stream, e salva o retorno da Função acima
        for idx, row in batch.iterrows():
            info_id = row['rating_id']
            info = get_info(info_id)
            for col in cols_to_add:
                df.loc[idx, col] = info.get(col, None)

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_info) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)  # para respeitar limite da API

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_info(df_combinado)

#Salvar df para csv
#df_combinado.to_csv(os.path.join(caminho_tabelas, config["base2"]), index=False, encoding='utf-8-sig')

df_combinado

In [ ]:
# OK tabelas Base para Relacional para o Rating - não precisa de API, só retorna em linhas separadas as ids

#Tabela 7 de Release Dates
#tabela_id = 7

#Seleção de Config a partir de tabela 7
#config = tabelas_config[tabela_id]

# Leitura dos arquivos
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["base2"]), sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_base
df_base = df_base.drop(columns=['rating_category'])

#Para o df_base, pode ser que tenham ID_IGDB vazios (já consultados e salvos previamente).
#É aplicado a função lambda para não necessitar reconsulta de API, preenchendo o vazio por 'NA'
df_base['rating_id'] = df_base['rating_id'].apply(lambda x: str(x) if pd.notna(x) else 'NA')
df_base['rating_content_descriptions'] = df_base['rating_content_descriptions'].apply(lambda x: str(x) if pd.notna(x) else 'NA')

# Remove duplicados de base e atualizado
df_base = df_base.drop_duplicates(subset='rating_content_descriptions', keep='first')

#Faz a cópia para evitar perda de informações no df
df_base = df_base.copy()

#-------------------------------------------------------------------------------------------------------------------------#

# Função para converter uma string para lista
# Garante que a coluna config["colunas"] esteja no formato correto (lista)
def safe_literal_eval(value):
    try:
        # Tenta converter a string para uma lista
        return ast.literal_eval(value) if isinstance(value, str) else value
    except (ValueError, SyntaxError):
        # Se der erro, retorna como uma lista vazia ou outro valor default
        print(f"[Aviso] Valor inválido para conversão: {value}")
        return []

#-------------------------------------------------------------------------------------------------------------------------#

# Função para expandir a coluna 'id_informação' em várias linhas
def expand_genre_rows(df, coluna_df):
    # Cria nova lista de linhas para salvar informação provisoriamente
    expanded_rows = []
    for _, row in df.iterrows():
        # Para cada linha, expandir os valores de coluna_df
        for genre in row[coluna_df]:
            expanded_rows.append({
                'rating_id': row['rating_id'],
                coluna_df: genre
            })

    # Cria um novo DataFrame com as linhas expandidas
    expanded_df = pd.DataFrame(expanded_rows)
    return expanded_df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada de função
df_base['rating_content_descriptions'] = df_base['rating_content_descriptions'].apply(safe_literal_eval)

# Chamada de função
df_expanded = expand_genre_rows(df_base, 'rating_content_descriptions')

#Salvar df para csv
#df_expanded.to_csv(os.path.join(caminho_tabelas, config["base3"]), index=False, encoding='utf-8-sig')

df_expanded

In [ ]:
#API IGDB - IGDB - Tabela Dimensional Age Rating - Part 3

#Tabela 7 de Release Dates
#tabela_id = 7

#Seleção de Config a partir de tabela 7
#config = tabelas_config[tabela_id]

#Leitura dos arquivos de tabelas relacionais e dimensionais
df_atualizado = pd.read_csv(os.path.join(caminho_tabelas, config["base2"]), sep=",", encoding='utf-8-sig')
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["dim"]), sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado
df_atualizado = df_atualizado.drop(columns=['rating_id', 'rating_content_descriptions'])

#Para garantir a duplicata, agrupa os jogos por ID da Data, depois remove coluna gerada
df_atualizado = df_atualizado.groupby(['rating_category']).size().reset_index(name='Count')
df_atualizado = df_atualizado.drop(columns=['Count'])

#Renomeia a coluna de rating para informação como ID
df_atualizado = df_atualizado.rename(columns={'rating_category': 'rating_category_id'})

# Adiciona colunas vazias para consulta depois
cols_to_add = ['rating', 'organization']
df_atualizado.loc[:, cols_to_add] = None

# Converte para string para evitar problemas
df_atualizado['rating_category_id'] = df_atualizado['rating_category_id'].astype(int).astype(str)
df_base['rating_category_id'] = df_base['rating_category_id'].astype(int).astype(str)
df_base['rating'] = df_base['rating'].apply(lambda x: str(x) if pd.notna(x) else 'NA')

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='rating_category_id', keep='first')

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = df_combinado.copy()

#Transforma as informações para a consulta API
df_combinado['rating'] = df_combinado['rating'].astype('string')  # ou 'object' para strings
df_combinado['organization'] = df_combinado['organization'].astype('Int64')  # Para inteiros com NA

#-------------------------------------------------------------------------------------------------------------------------#

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar as informações da API
def get_info(info_id):
    #Checa se o ID já tem informação ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if info_id in cache_info:
        return cache_info[info_id]

    #Definição do body da URL com a ID
    body = f"""
    where id = {info_id};
    fields rating, organization;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {info_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(config["url_base_dim_2"], headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:

            #200 deu certo, salva resposta JSON na variável info_data (dicionário)
            info_data = response.json()

            #Verificação da lista "data" no dicionário "info_data"
            if info_data:
                #Salva a informação ao acessar o primeiro item da lista ([0])
                info = info_data[0]

                #Caso não encontre, salva como 'NA'
                info_dict = {
                    'organization': info.get('organization'),
                    'rating': str(info.get('rating')) if info.get('rating') not in [None, []] else 'NA'
                }

                #Salva no dicionário cache
                cache_info[info_id] = info_dict

                #Retorno da função
                return info_dict

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Resposta 200, mas sem dados para o ID {info_id}")
                info_dict = {
                    'rating': 'NA', # Armazenar 'NA' se não houver informação
                    'organization': None  # Usando None para valores ausentes numéricos
                }
                cache_info[info_id] = info_dict
                return info_dict

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {info_id} - Status: {response.status_code}")
            info_dict = {
                'rating': 'Erro',  # Armazenar 'Erro' se a API falhar
                'organization': None  # Usando None para valores ausentes numéricos
            }
            cache_info[info_id] = info_dict
            return info_dict

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {info_id}: {e}")
        info_dict = {
            'rating': 'Erro',  # Armazenar 'Erro' se houver erro na consulta
            'organization': None  # Usando None para valores ausentes numéricos
        }
        cache_info[info_id] = info_dict
        return info_dict

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_info(df):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_info = df[df['rating'].isna() | (df['rating'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_info.empty:
        print("Nenhum date novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5
    #len(df_faltando_info)

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_faltando_info), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_faltando_info.iloc[i:i + batch_size]

        # Para cada linha do batch, pega o ID do Stream, e salva o retorno da Função acima
        for idx, row in batch.iterrows():
            info_id = row['rating_category_id']
            info = get_info(info_id)
            for col in cols_to_add:
                df.loc[idx, col] = info.get(col, None)

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_info) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)  # para respeitar limite da API

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_info(df_combinado)

#Salvar df para csv
#df_combinado.to_csv(os.path.join(caminho_tabelas, config["dim"]), index=False, encoding='utf-8-sig')

df_combinado

In [ ]:
#API IGDB - IGDB - Tabela Dimensional Age Rating - Part 4

#Tabela 7 de Release Dates
#tabela_id = 7

#Seleção de Config a partir de tabela 7
#config = tabelas_config[tabela_id]

#Leitura dos arquivos de tabelas relacionais e dimensionais
df_atualizado = pd.read_csv(os.path.join(caminho_tabelas, config["base3"]), sep=",", encoding='utf-8-sig')
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["dim2"]), sep=",", encoding='utf-8-sig')

#Para garantir a duplicata, agrupa os jogos por ID da Data, depois remove coluna gerada
df_atualizado = df_atualizado.groupby(['rating_content_descriptions']).size().reset_index(name='Count')
df_atualizado = df_atualizado.drop(columns=['Count'])

#Renomeia a coluna de rating para informação como ID
df_atualizado = df_atualizado.rename(columns={'rating_content_descriptions': 'rating_content_descriptions_id'})

# Converte para string para evitar problemas
df_atualizado['rating_content_descriptions_id'] = df_atualizado['rating_content_descriptions_id'].astype(str)
df_base['rating_content_descriptions_id'] = df_base['rating_content_descriptions_id'].astype(str)

#Somente na primeira vez que salva
#df_base['description'] = ''

#Para o df_base, pode ser que tenham ID vazios (já consultados e salvos previamente).
#É aplicado a função lambda para não necessitar reconsulta de API, preenchendo o vazio por 'NA'
df_base['description'] = df_base['description'].apply(lambda x: str(x) if pd.notna(x) else 'NA')

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='rating_content_descriptions_id', keep='first')

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = df_combinado.copy()

#Transforma as informações para a consulta API
df_combinado['description'] = df_combinado['description'].astype('string')  # ou 'object' para strings


#-------------------------------------------------------------------------------------------------------------------------#

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar as informações da API
def get_info(info_id):
    #Checa se o ID já tem informação ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if info_id in cache_info:
        return cache_info[info_id]

    #Definição do body da URL com a ID
    body = f"""
    where id = {info_id};
    fields description;
    limit 1;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {info_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(config["url_base_dim_3"], headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:
            #200 deu certo, salva resposta JSON na variável info_data (dicionário)
            game_data = response.json()

            #Verificação da lista "data" no dicionário "info_data"
            if isinstance(game_data, list) and len(game_data) > 0:
                #Salva a informação ao acessar o primeiro item da lista ([0])
                #Caso não encontre, salva como 'NA'
                info = str(game_data[0].get('description') or 'NA')
                print(f"Info encontrado para {info_id}: {info}")

                #Retorno da função
                return info

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"Nenhum dado retornado para {info_id}")
                return 'NA'  # Armazenar 'NA' se não houver dados

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"Erro ao acessar API para {info_id}: {response.status_code}")
            return 'Erro'  # Armazenar 'Erro' se a API falhar

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"Exceção ao consultar API para {info_id}: {e}")
        return 'Erro'  # Armazenar 'Erro' se houver erro na consulta

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_info(df):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_to_update = df[df['description'].isna() | (df['description'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_to_update.empty:
        print("Todos os jogos já têm description. Nenhuma chamada de API necessária.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5
    # len(df_to_update)

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_to_update), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_to_update.iloc[i:i + batch_size].copy()

        # Para cada linha do batch, pega o ID da informação, e salva o retorno da Função acima
        batch['description'] = batch['rating_content_descriptions_id'].apply(get_info)
        df.loc[batch.index, 'description'] = batch['description']

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_to_update) - 1) // batch_size + 1}")
        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_info(df_combinado)

#Salvar df para csv
#df_combinado.to_csv(os.path.join(caminho_tabelas, config["dim2"]), index=False, encoding='utf-8-sig')

df_combinado

# Tabela Summary

In [ ]:
#OK API IGDB - Summary

#Tabela 8 de Summary
tabela_id = 8

#Seleção de Config a partir de tabela 8
config = tabelas_config[tabela_id]

#Leitura dos arquivos de da tabela Twitch IGDB Database e Dimensão de Summary
df_atualizado = pd.read_csv(twitch_database_igdb, sep=",", encoding='utf-8-sig')
df_base = pd.read_csv(os.path.join(caminho_tabelas, config["dim"]), sep=",", encoding='utf-8-sig')

# Remove colunas desnecessárias do df_atualizado
df_atualizado = df_atualizado.drop(columns=['ID_Twitch', 'NomedoJogo'])

#Para garantir a duplicata, agrupa os jogos por ID da Data, depois remove coluna gerada
df_atualizado = df_atualizado.groupby(['ID_IGDB']).size().reset_index(name='Count')
df_atualizado = df_atualizado.drop(columns=['Count'])

# Adiciona colunas vazias para consulta depois
cols_to_add = ['name', 'summary']
df_atualizado.loc[:, cols_to_add] = None

# Converte para string para evitar problemas
df_atualizado['ID_IGDB'] = df_atualizado['ID_IGDB'].astype(int).astype(str)
df_base['ID_IGDB'] = df_base['ID_IGDB'].astype(int).astype(str)

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = pd.concat([df_base, df_atualizado], ignore_index=True)
df_combinado = df_combinado.drop_duplicates(subset='ID_IGDB', keep='first')

# Junta dados antigos + novos no df_combinado sem duplicatas
df_combinado = df_combinado.copy()

#Transforma as informações para a consulta API
df_combinado['name'] = df_combinado['name'].astype('string')  # ou 'object' para strings
df_combinado['summary'] = df_combinado['summary'].astype('string')  # ou 'object' para strings

#-------------------------------------------------------------------------------------------------------------------------#

# Client-ID e Token de Autenticação na definição dos headers
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}'
}

#Dicionário vazio para salvar a resposta da API
cache_info = {}

#-------------------------------------------------------------------------------------------------------------------------#

# Função para pegar as informações da API
def get_game_info(game_id):
    #Checa se o ID já tem informação ou não (certeza do funcionamento para remoção de IDs duplicados, principalmente os consultados pela API), utilizando o dicionário
    if game_id in cache_info:
        return cache_info[game_id]

    #Definição do body da URL com a ID
    body = f"""
    where id = {game_id};
    fields name, summary;
    limit 1;
    """

    #Informa o usuário se está fazendo a chamada da API para o ID solicitado
    print(f"Chamando API para o ID: {game_id}")

    #Dependendo da resposta da API:
    try:
        #Salva resposta de status da API para conferência
        response = requests.post(config["url_base_dim"], headers=headers, data=body)

        #Checa qual status da resposta
        if response.status_code == 200:

            #200 deu certo, salva resposta JSON na variável info_data (dicionário)
            game_data = response.json()

            #Verificação da lista "data" no dicionário "info_data"
            if game_data:

                #Salva a informação ao acessar o primeiro item da lista ([0])
                game_info = game_data[0]

                #Caso não encontre, salva como 'NA'
                game_info_dict = {
                    'name': game_info.get('name') if game_info.get('name') else 'NA',
                    'summary': game_info.get('summary')
                }

                #Salva no dicionário cache
                cache_info[game_id] = game_info_dict

                #Retorno da função
                return game_info_dict

            #Caso não tenha nada (resposta vazia) em "data"
            else:
                print(f"[Aviso] Resposta 200, mas sem dados para o ID {game_id}")
                game_info_dict = {
                    'name': 'NA',  # Armazenar 'NA' se não houver informação
                    'summary': 'NA'
                }
                cache_info[game_id] = game_info_dict
                return game_info_dict

        #Falha no acesso da API (401 - Token OAuth não válido)
        #Sai do Programa sem sobreescrever no database
        elif response.status_code == 401:
            try:
                erro_json = response.json()
                mensagem = erro_json.get("message", "Token inválido ou expirado.")
            except ValueError:
                mensagem = "Erro 401: Não autorizado. (Resposta não é JSON)"
            print(f"[Erro 401] Acesso não autorizado")
            print(f"Mensagem da API: {mensagem}")
            print("Encerrando o programa. Verifique se o token OAuth está correto ou ainda válido.")
            sys.exit(1)

        #Falha na consulta API, com o status para verificação
        else:
            print(f"[Erro] API falhou para o ID {game_id} - Status: {response.status_code}")
            game_info_dict = {
                'name': 'Erro',  # Armazenar 'Erro' se a API falhar
                'summary': 'Erro'
            }
            cache_info[game_id] = game_info_dict
            return game_info_dict

    #Falha na consulta da API por outro motivo
    except Exception as e:
        print(f"[Erro de exceção] Falha ao consultar API para o ID {game_id}: {e}")
        game_info_dict = {
            'name': 'Erro',  # Armazenar 'Erro' se houver erro na consulta
            'summary': 'Erro'
        }
        cache_info[game_id] = game_info_dict
        return game_info_dict

#-------------------------------------------------------------------------------------------------------------------------#

# Função para atualizar o DataFrame
def update_game_info(df):
    #Copia em um novo df apenas os ID que possuem Id vazia para consulta na API
    #Confere se é um valor ausente ou se está vazio (mesmo sem espaços)
    df_faltando_info = df[df['name'].isna() | (df['name'].astype(str).str.strip() == '')].copy()

    #Se o novo df estiver vazio, não há informações para consultar
    if df_faltando_info.empty:
        print("Nenhum jogo novo para consultar na API.")
        return df

    #Definição de "lotes" para consulta à API, para não sobrecarregar as consultas
      #Quantidade de informações processados por iteração
    batch_size = 5
    #len(df_faltando_info)

    #Processamento das consultas por lotes de todo o df a ser consultado
    for i in range(0, len(df_faltando_info), batch_size):
        #Define o batch conforme posição das linhas por indexação (0 até 5 - sem incluir o 6)
        batch = df_faltando_info.iloc[i:i + batch_size]

        # Para cada linha do batch, pega o ID do Stream, e salva o retorno da Função acima
        for idx, row in batch.iterrows():
            game_id = row['ID_IGDB']
            info = get_game_info(game_id)
            for col in cols_to_add:
                df.loc[idx, col] = info.get(col, None)

        #Informa usuário qual lote (inteiro) está sendo processado
        print(f"Processando o lote {i // batch_size + 1} de {(len(df_faltando_info) - 1) // batch_size + 1}")

        #Pausa de 5s para evitar sobrecarregar a API e evitar limites de taxa (rate limits)
        time.sleep(5)  # para respeitar limite da API

    return df

#-------------------------------------------------------------------------------------------------------------------------#

# Chamada da principal função
df_combinado = update_game_info(df_combinado)

#Salvar df para csv
#df_combinado.to_csv(os.path.join(caminho_tabelas, config["dim"]), index=False, encoding='utf-8-sig')

df_combinado
